In [121]:
class Mat:
    """Implementsção de uma matris
    """
    def __init__(self, rows, cols):
        #print(f"construtor do Mat invocado com {rows} e {cols}")
        self.mat = [[0 for c in range(cols)]
                    for r in range(rows)]
    def numRows (self): return len(self.mat)
    def numCols (self): return len(self.mat[0])
    def __str__(self):
        "Devolve a matriz como uma string"
        return '\n'.join(' '.join(str(val) for val in row)
                         for row in self.mat)
    # def __repr__(self):
    #     "Utilizado no repl quando se pede para ver a matriz"
    #     return str(self)
    def __getitem__ (self, n):
        "Interface para a indexação []"
        return self.mat[n]

In [122]:
m = Mat(3,4)
m[0][1] = -5
m

In [132]:
class DotPlot(Mat):

    def __init__(self, seq1, seq2):
        super().__init__(len(seq1), len(seq2))
        self.s1 = seq1
        self.s2 = seq2
        self.dims = (len(seq1), len(seq2))

    def get_s1(self): 
        return self.s1
  
    def get_s2(self): 
        return self.s2

    # def __repr__(self):
    #     import io
    #     with io.StringIO() as s:
    #         print(' ', *list(self.s2), file = s)
    #         for x, L in zip(self.s1, self.mat):
    #             print(x, *L, file = s)
    #         return s.getvalue()


    def simple_dotplot(self):
        for L, x1 in enumerate(self.s1):
            for C, x2 in enumerate(self.s2):
                self.mat[L][C] = '*' if x1 == x2 else ' '
        import io
        with io.StringIO() as s:
            print(' ', *list(self.s2), file = s)
            for x, L in zip(self.s1, self.mat):
                print(x, *L, file = s)
            return print(s.getvalue())


    def extended_dotplot(self , window_size = 1, stringency = 1):     # Inspirado Prof Miguel Rocha
        start = int(window_size/2)  # isto para começar "centrado" e não haver erros de limites
        for L in range (start, len(self.s1) - start):
            for C in range(start, len(self.s2) - start):
                matches = 0
                l = C - start
                for k in range (L-start, L+start +1):
                    if self.s1[k] == self.s2[l]:
                        matches += 1
                    l += 1
                if matches >= stringency:
                    self.mat[L][C]= "*"
                else:
                    self.mat[L][C]= " "
        import io
        with io.StringIO() as s:
            print(' ', *list(self.s2), file = s)
            for x, L in zip(self.s1, self.mat):
                print(x, *L, file = s)
            return print(s.getvalue())

In [136]:
m = DotPlot("CGATATTAACGATT", "CGATATTAACGATT")
m.extended_dotplot(4,3)


  C G A T A T T A A C G A T T
C 0 0 0 0 0 0 0 0 0 0 0 0 0 0
G 0 0 0 0 0 0 0 0 0 0 0 0 0 0
A 0 0 *                 * 0 0
T 0 0   *                 0 0
A 0 0     *             * 0 0
T 0 0       *             0 0
T 0 0         *           0 0
A 0 0           *         0 0
A 0 0             *       0 0
C 0 0               *     0 0
G 0 0                 *   0 0
A 0 0 *   *             * 0 0
T 0 0 0 0 0 0 0 0 0 0 0 0 0 0
T 0 0 0 0 0 0 0 0 0 0 0 0 0 0

